In [1]:
# import Libraries
# Built-in Python libraries
import os # functions for interacting w operating system

# 3rd-party libraries
import numpy as np # module for low-level scientific computing
import easygui
import pandas as pd
import itertools
import glob
from datetime import date

In [2]:
#Define a padding function
def boolean_indexing(v, fillval=np.nan):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.full(mask.shape,fillval)
    out[mask] = np.concatenate(v)
    return out

In [3]:
file_path = easygui.diropenbox(msg = 'Choose a directory with lick data')
os.chdir(file_path)
for i in os.listdir():
    print(i)
    if 'txt' in i:
        text_file = i
file_name = os.path.join(file_path, text_file)

0723KM35_TEST1.ms8.txt
0723KM35_TEST1.pkl


In [4]:
file_input = open(file_name)

In [5]:
lines = file_input.readlines()
len(lines)

131

In [6]:
Detail_Dict = {'FileName': None,\
               'StartDate': None,\
               'StartTime': None,\
               'Animal': None,\
               'MAXFLick': None,\
               'Trials': None,\
               'LickDF': None,\
               'LatencyMatrix': None}

In [7]:
dick_backup = Detail_Dict.copy()

In [8]:
#Extract file name and store
Detail_Dict['FileName'] = file_name[file_name.rfind('/')+1:]

#Store details in dictionary and construct dataframe	
for i in range(len(lines)):
    if "Start Date" in lines[i]:
        Detail_Dict['StartDate'] = lines[i].split(',')[-1][:-1].strip()
    if "Start Time" in lines[i]:
        Detail_Dict['StartTime'] = lines[i].split(',')[-1][:-1]
    if "Animal ID" in lines[i]:
        Detail_Dict['Animal'] = lines[i].split(',')[-1][:-1]		
    if "Max Wait" in lines[i]:
        Detail_Dict['MAXFLick'] = lines[i].split(',')[-1][:-1]			
    if "Max Number" in lines[i]:
        Detail_Dict['Trials'] = lines[i].split(',')[-1][:-1]			
    if "PRESENTATION" and "TUBE" in lines[i]:		
        ID_line = i
    if len(lines[i].strip()) == 0:		
        Trial_data_stop = i		
        if ID_line > 0 and Trial_data_stop > 0:
            #Create dataframe
            df = pd.DataFrame(columns=lines[ID_line].split(','),\
                      data=[row.split(',') for row in \
                     lines[ID_line+1:Trial_data_stop]])
            
            #Remove spaces in column headers (caused by split)
            df.columns = df.columns.str.replace(' ', '')
            
            #Set concentrations to 0 if concentration column blank
            df['CONCENTRATION']=df['CONCENTRATION'].str.strip()
            df['CONCENTRATION'] = df['CONCENTRATION'].apply(lambda x: 0 if x == '' else x)

            #Convert specific columns to numeric
            df["SOLUTION"] = df["SOLUTION"].str.strip()
            df[["PRESENTATION","TUBE","CONCENTRATION","LICKS","Latency"]] = \
                df[["PRESENTATION","TUBE","CONCENTRATION","LICKS","Latency"]].apply(pd.to_numeric)
            
            #Add in identifier columns
            df.insert(loc=0, column='Animal', value=Detail_Dict['Animal'])
            df.insert(loc=3, column='Trial_num', value='')
            df['Trial_num'] = df.groupby('TUBE').cumcount()+1
            
            #Store in dataframe
            Detail_Dict['LickDF'] = df		
            
            #Grab all ILI data, pad with NaNs to make even matrix
            #Store in dictionary (TrialXILI count)
            Detail_Dict['LatencyMatrix'] = boolean_indexing([row.split(',')\
                                              for row in lines[Trial_data_stop+1:]])


In [14]:
# clean dataframe
# make column names all lower case
col_names = [i.lower() for i in df.columns]
df.columns = col_names
# drop unused column
df = df.drop('openerror', axis=1)
df = df.drop('closeerror\n', axis=1)
df.columns


Index(['animal', 'presentation', 'tube', 'trial_num', 'concentration',
       'solution', 'ipi', 'length', 'licks', 'latency', 'licks'],
      dtype='object')

In [15]:
# reading in ilis and save to dataframe
ilis = []
#lines_list = lines[Trial_data_stop+1:]
for i in range(Trial_data_stop+1, len(lines)):
    this_line = lines[i].split(',')[1:]
    num_line = [int(i) for i in this_line]
    ilis.append(num_line)
ilis[0]

[651, 656, 726, 130, 626, 138, 139, 131, 137, 136, 130, 126, 120, 130]

In [17]:
if np.max(df.presentation) == len(ilis):
    df['ilis'] = ilis
else:
    print('Something wrong with the itis reading! please check your text files')

In [25]:
df_name = Detail_Dict['Animal'].strip() + '_' + Detail_Dict['StartDate'].replace('/', '')
df.to_pickle(os.path.join(file_path, f"{df_name}.pkl"))

In [26]:
df

,animal,presentation,tube,trial_num,concentration,solution,ipi,length,licks,latency,licks,ilis
0,KM35,1,1,1,0,CITRIC_ACID,10,10,15,576,"[651, 656, 726, 130, 626, 138, 139, 131, 137, ...","[651, 656, 726, 130, 626, 138, 139, 131, 137, ..."
1,KM35,2,4,1,0,SUCROSE,10,10,21,7187,"[201, 412, 231, 320, 4106, 128, 128, 130, 128,...","[201, 412, 231, 320, 4106, 128, 128, 130, 128,..."
2,KM35,3,5,1,0,WATER,10,10,0,60000,[],[]
3,KM35,4,2,1,0,QHCL,10,10,17,23876,"[149, 129, 131, 146, 1046, 232, 61, 282, 86, 1...","[149, 129, 131, 146, 1046, 232, 61, 282, 86, 1..."
4,KM35,5,3,1,0,NACL,10,10,0,60000,[],[]
5,KM35,6,2,2,0,QHCL,10,10,0,60000,[],[]
6,KM35,7,4,2,0,SUCROSE,10,10,17,26765,"[392, 418, 123, 142, 152, 147, 121, 145, 3674,...","[392, 418, 123, 142, 152, 147, 121, 145, 3674,..."
7,KM35,8,5,2,0,WATER,10,10,0,60000,[],[]
8,KM35,9,3,2,0,NACL,10,10,88,29232,"[27, 96, 125, 128, 133, 110, 116, 130, 125, 37...","[27, 96, 125, 128, 133, 110, 116, 130, 125, 37..."
9,KM35,10,1,2,0,CITRIC_ACID,10,10,11,2231,"[251, 132, 3383, 146, 141, 130, 140, 126, 127,...","[251, 132, 3383, 146, 141, 130, 140, 126, 127,..."
